### Clean and recode SECC data

In [1]:
import pandas as pd
import gc

#### Prepare state <=> numeric mapping

In [2]:
df = pd.read_csv('/opt/data/secc/v2/secc_deduped.csv.gz', usecols=['state'])
df

,state
0,ARUNACHAL PRADESH
1,ARUNACHAL PRADESH
2,ARUNACHAL PRADESH
3,ARUNACHAL PRADESH
4,ARUNACHAL PRADESH
...,...
140198227,WEST BENGAL
140198228,WEST BENGAL
140198229,WEST BENGAL
140198230,WEST BENGAL


In [3]:
df.state = df.state.str.lower()

In [4]:
sdf = df.groupby(['state']).agg({'state': 'count'})

In [5]:
sdf.rename(columns={'state': 'count'}, inplace=True)
sdf.reset_index(inplace=True)

In [6]:
cat = sdf.state.astype('category').cat

In [7]:
states_map = dict(zip(cat.categories, cat.codes))

In [8]:
pd.DataFrame.from_dict(states_map, orient='index', columns=['code']).to_csv('secc_state.csv', index_label='state')

In [9]:
d = pd.read_csv('secc_state.csv').to_dict(orient='list')
states_map = dict(zip(d['state'], d['code']))
states_map

{'arunachal pradesh': 0,
 'assam': 1,
 'bihar': 2,
 'chhattisgarh': 3,
 'gujarat': 4,
 'haryana': 5,
 'kerala': 6,
 'madhya pradesh': 7,
 'maharashtra': 8,
 'mizoram': 9,
 'nagaland': 10,
 'odisha': 11,
 'punjab': 12,
 'rajasthan': 13,
 'sikkim': 14,
 'tamilnadu': 15,
 'uttar pradesh': 16,
 'uttarakhand': 17,
 'west bengal': 18}

In [10]:
social_cats_map = {'st': 't', 'sc': 'c', 'other': 'o'}

In [15]:
header = True
count = 0
for df in pd.read_csv('/opt/data/secc/v2/secc_deduped.csv.gz', usecols=['state', 'head_of_hh', 'social_cat', 'age', 'gender'], dtype=str, chunksize=1000000):
    count += 1
    print(count)
    df.state = df.state.str.lower()
    df.gender = df.gender.str.lower()
    df.social_cat = df.social_cat.str.lower()
    df['name'] = df.head_of_hh.str.split('\\').str[0]
    del df['head_of_hh']
    df.name = df.name.str.lower()
    df.dropna(subset=['age'], inplace=True)
    df.drop(df[~df.age.str.contains('^[0-9\.]+$')].index, inplace=True)
    df['birth_year'] = 2011 - df.age.astype(float)
    df.birth_year = df.birth_year.astype(int)
    df.state = df.state.apply(lambda c: states_map[c])
    df.social_cat = df.social_cat.apply(lambda c: social_cats_map[c])
    df.name = df.name.str.replace('_', ' ')
    df.name = df.name.str.strip()
    df.drop(df[df.name==' '].index, inplace=True)
    df.drop(df[df.name==''].index, inplace=True)
    #df.drop(df[~df.name.str.isalpha()].index, inplace=True)
    df['name_wc'] = df.name.apply(lambda c: len(c.split()))
    df['last_name'] = None
    df.loc[df.name_wc >= 2, 'last_name'] = df.name.apply(lambda c: c.split()[-1])
    #df.loc[df.last_name.notnull() & (df.last_name.str.len() <= 2), 'last_name'] = None
    del df['name']
    del df['name_wc']
    del df['age']
    df.to_csv('/opt/data/secc/v2/secc_state_ln_sex_social_cat_birthyear_recode.csv', mode='a', header=header, index=False)
    header = False

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141


In [16]:
!gzip /opt/data/secc/v2/secc_state_ln_sex_social_cat_birthyear_recode.csv